# Setup


In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# Tools

In [2]:
from tool_evaluator_agent.tools import ScrapingFishTool

In [20]:
import os
SCRAPING_FISH_API_KEY = os.getenv("SCRAPING_FISH_API_KEY")


In [3]:
content = ScrapingFishTool().run('https://www.orulo.com.br')

Using Tool: Scrape website content


# Models

## Google Gemini

In [2]:
import os
from google.oauth2 import service_account
from langchain_google_genai import ChatGoogleGenerativeAI

# Set the path to the service account JSON key file
service_account_file = os.getenv("GOOGLE_SERVICE_ACCOUNT_FILE")

# Authenticate using the service account key
credentials = service_account.Credentials.from_service_account_file(service_account_file)


/Users/aolchik/mambaforge/envs/crewai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash',
                            verbose=True,
                            temperature=0.5,
                            credentials=credentials)

I0000 00:00:1724249811.931081 19971561 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1724249811.931865 19971561 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [7]:
print(type(llm))

<class 'langchain_google_genai.chat_models.ChatGoogleGenerativeAI'>


In [8]:
isinstance(llm, ChatGoogleGenerativeAI)

True

In [13]:
llm.model.split('/')[-1]

'gemini-1.5-flash'

In [4]:
llm.invoke("Sing a ballad of LangChain.")

AIMessage(content="(Verse 1)\nIn the realm of code, where data flows,\nA tool was born, LangChain, it goes.\nWith chains of thought, it weaves its spell,\nTo unlock knowledge, secrets to tell.\n\n(Chorus)\nOh, LangChain, LangChain, a symphony of code,\nConnecting minds, a path we've strode.\nWith prompts and chains, we tap the well,\nOf language models, stories to tell.\n\n(Verse 2)\nFrom LLM's power, we draw our might,\nTo answer questions, day and night.\nWith prompts we guide, with chains we bind,\nA tapestry of words, for all to find.\n\n(Chorus)\nOh, LangChain, LangChain, a symphony of code,\nConnecting minds, a path we've strode.\nWith prompts and chains, we tap the well,\nOf language models, stories to tell.\n\n(Verse 3)\nThrough agents and tools, it takes its stand,\nTo automate tasks, with a helping hand.\nFrom summarization to text creation,\nLangChain empowers, with innovation.\n\n(Chorus)\nOh, LangChain, LangChain, a symphony of code,\nConnecting minds, a path we've strode.

## OpenAI

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o',
                  verbose=True,
                  temperature=0)

In [ ]:
?ChatOpenAI

## LiteLLM

In [2]:
import os
from google.oauth2 import service_account
from langchain_google_genai import ChatGoogleGenerativeAI

# Set the path to the service account JSON key file
service_account_file = os.getenv("GOOGLE_SERVICE_ACCOUNT_FILE")

# Authenticate using the service account key
credentials = service_account.Credentials.from_service_account_file(service_account_file)

/Users/aolchik/mambaforge/envs/crewai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain_community.chat_models import ChatLiteLLM
chat = ChatLiteLLM(model="gpt-3.5-turbo")

In [7]:
type(chat)

langchain_community.chat_models.litellm.ChatLiteLLM

In [15]:
chat.model_name = chat.model

In [17]:
chat.model_name

'gpt-3.5-turbo'

In [16]:
hasattr(chat, "model_name")

True

## Ollama

In [53]:
from langchain_community.llms import Ollama

llm = Ollama(model='invalid')

## LLM Wrapper


In [2]:
from helpers.llm_wrapper import ModelParams

p = ModelParams()


In [4]:
p.use_cache

True

# Observability


## Helicone

In [33]:
params = {
    "model": 'gpt-3.5-turbo',
    "temperature": 0,
    "verbose": True,
}

params["base_url"] = "https://oai.helicone.ai/v1"

In [34]:
import os
headers = {
    "Helicone-Auth": f"Bearer {os.getenv('HELICONE_API_KEY')}",
}

In [35]:
headers

{'Helicone-Auth': 'Bearer pk-helicone-itv4c7y-xxgecla-wegp4bq-ntrcs6a'}

In [36]:
params["default_headers"] = headers

In [37]:
params

{'model': 'gpt-3.5-turbo',
 'temperature': 0,
 'verbose': True,
 'base_url': 'https://oai.helicone.ai/v1',
 'default_headers': {'Helicone-Auth': 'Bearer pk-helicone-itv4c7y-xxgecla-wegp4bq-ntrcs6a'}}

In [49]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(**params)

In [50]:
llm.invoke("Sing a ballad of LangChain.")

AIMessage(content="In the land of LangChain, a tale unfolds\nOf heroes and villains, of legends untold\nA kingdom divided, by power and greed\nBut a hero arises, to fulfill a great need\n\nLangChain, oh LangChain, a land of strife\nWhere darkness and light, battle for life\nBut in the shadows, a hero stands tall\nTo bring peace and justice, to one and all\n\nWith sword in hand, and heart of gold\nLangChain's hero, brave and bold\nFights for the innocent, fights for the weak\nTo bring an end, to the darkness that seeks\n\nThrough battles and trials, the hero prevails\nWith courage and honor, their quest never fails\nLangChain is saved, by their selfless deed\nA legend forever, in LangChain's creed\n\nSo raise a toast, to LangChain's hero true\nTheir story will live on, forever anew\nIn the hearts of the people, their legend will reign\nIn the land of LangChain, where heroes remain.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 209, 'promp

In [40]:
from helpers.tracer import TracerFactory

tracer = TracerFactory.get_tracer()

In [42]:
from helpers.tracer import TracerAnnotation
tracer.provider = 'helicone'
tracer.annotation = TracerAnnotation(app='tool_evaluator_agent')

In [43]:
proxy_config = tracer.get_proxy_config()
proxy_config

LLMProxyConfig(url='https://oai.helicone.ai/v1', headers={'Helicone-Auth': 'Bearer pk-helicone-itv4c7y-xxgecla-wegp4bq-ntrcs6a', 'Helicone-Cache-Enabled': 'true', 'Helicone-Property-Session': '45b4bdb6-4803-4de2-86e1-7776ebf05344', 'Helicone-Property-App': 'tool_evaluator_agent'})

In [48]:
params

{'model': 'gpt-3.5-turbo',
 'temperature': 0,
 'verbose': True,
 'base_url': 'https://oai.helicone.ai/v1',
 'default_headers': {'Helicone-Auth': 'Bearer pk-helicone-itv4c7y-xxgecla-wegp4bq-ntrcs6a',
  'Helicone-Cache-Enabled': 'true',
  'Helicone-Property-Session': '45b4bdb6-4803-4de2-86e1-7776ebf05344',
  'Helicone-Property-App': 'tool_evaluator_agent'}}

In [47]:
params["base_url"] = proxy_config.url
params["default_headers"] = proxy_config.headers

In [2]:
__name__

'__main__'

In [3]:
## Tracer

In [12]:
from helpers.tracer import TracerFactory, TracerAnnotation
tracer = TracerFactory.get_tracer()

In [10]:
tracer

Tracer(provider='helicone', annotation=None)

In [7]:
tracer.provider = 'helicone'

In [8]:
tracer

Tracer(provider='helicone', annotation=None)

In [13]:
tracer.get_proxy_config()

LLMProxyConfig(url='https://oai.helicone.ai/v1', headers={'Helicone-Auth': 'Bearer pk-helicone-itv4c7y-xxgecla-wegp4bq-ntrcs6a', 'Helicone-Cache-Enabled': 'true'})

## Logger

In [5]:
import logging

In [7]:
logger = logging.getLogger('tool-evaluator-agent')

In [9]:
provider = "Google"
logger.debug(f"Tracer ended with provider: {provider}")

2024-08-26 10:46:44,684 - tool-evaluator-agent - DEBUG - Tracer ended with provider: Google


In [ ]:
# Configure the root logger
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)


In [11]:
logger = logging.getLogger('tool-evaluator-agent')
provider = "Google"
logger.debug(f"Tracer ended with provider: {provider}")

2024-08-26 10:47:51,295 - tool-evaluator-agent - DEBUG - Tracer ended with provider: Google


In [12]:

# Create a common logger instance
logger = logging.getLogger('tool-evaluator-agent')

# Example: Adding a file handler
file_handler = logging.FileHandler('logs/app-debug.log')
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(
    logging.Formatter('%(asctime)s - %(name)s - '
                      '%(levelname)s - %(message)s'))
logger.addHandler(file_handler)

In [16]:
logger = logging.getLogger('tool-evaluator-agent')
provider = "Google"
logger.debug(f"Tracer ended with provider: {provider}")

2024-08-26 10:54:17,065 - tool-evaluator-agent - DEBUG - Tracer ended with provider: Google


In [14]:
type(logger)

logging.Logger

In [15]:
print(logger)

<Logger tool-evaluator-agent (DEBUG)>
